In [1]:
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

In [2]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [25]:
import numpy as np
import random
from numba import jit, njit

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection, DimensionAwareStrategy
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice, numba_random_gauss, numba_random_cauchy, numba_random_uniform
from pyMSOO.utils.Search import *

class model(AbstractModel.model):
    TOLERANCE = 1e-6
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, 
        mutation: Mutation.PolynomialMutation, 
        selection: Selection.ElitismSelection,
        dimension_strategy: DimensionAwareStrategy.AbstractDaS = DimensionAwareStrategy.NoDaS(),
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation,dimension_strategy, selection, *args, **kwargs)
    
    def fit(self, nb_generations, 
            nb_inds_each_task = 100, 
            nb_inds_max = 100,
            nb_inds_min = 20,
            evaluate_initial_skillFactor = True, 
            c = 0.06,
            *args, 
            **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # nb_inds_min
        if nb_inds_min is not None:
            assert nb_inds_each_task >= nb_inds_min
        else: 
            nb_inds_min = nb_inds_each_task

        self.rmp = np.full((len(self.tasks), len(self.tasks)), 0.3)
        # np.fill_diagonal(self.rmp, 0)

        # self.delta = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

        # self.s_rmp = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

        self.learningPhase = [LearningPhase(self.IndClass, self.tasks, t) for t in self.tasks]
        
        # initialize population
        self.population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )

        self.nb_inds_tasks = [nb_inds_each_task] * len(self.tasks)

        MAXEVALS = nb_generations * nb_inds_each_task * len(self.tasks)
        self.eval_k = [0] * len(self.tasks)
        epoch = 1
        
        D0 = self.calculateD(population = np.array([[ind.genes for ind in sub.ls_inds] for sub in self.population]), 
                            population_fitness = np.array([sub.getFitness() for sub in self.population]),
                            best = np.array([sub.__getBestIndividual__.genes for sub in self.population]),)

        while sum(self.eval_k) < MAXEVALS:
            self.population.update_rank()

            if sum(self.eval_k) >= epoch * nb_inds_each_task * len(self.tasks):
                # save history
                self.history_cost.append([ind.fcost for ind in self.population.get_solves()])
                self.render_process(epoch/nb_generations, ['Pop_size', 'Cost'], [[sum(self.nb_inds_tasks)], self.history_cost[-1]], use_sys= True)
                epoch += 1
            
            self.delta = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

            self.s_rmp = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

            # offsprings = self.reproduction(sum(self.nb_inds_tasks), self.population)

            # self.population = self.population + offsprings

            matingPool = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
                evaluate_initial_skillFactor = False
            )

            for idx in range(len(self.tasks)):
                
                idx_inds = np.argsort([ind.fcost for ind in self.population[idx].ls_inds])
                
                for i in idx_inds[:int(len(self.population[idx])/2)]:
                    matingPool.__addIndividual__(self.population[idx].ls_inds[i])

            offsprings = self.reproduction(len(self.population), matingPool)
            
            # merge and update rank
            self.population = matingPool + offsprings
            self.population.update_rank()
            
            # selection
            self.nb_inds_tasks = [int(
                int(max((nb_inds_min - nb_inds_max) * (sum(self.eval_k)/MAXEVALS) + nb_inds_max, nb_inds_min))
            )] * len(self.tasks)
            self.selection(self.population, self.nb_inds_tasks)

            # update operators
            self.crossover.update(population = self.population)
            self.mutation.update(population = self.population)
            self.dimension_strategy.update(population = self.population)

            self.updateRMP(c)

            self.phaseTwo(D0)

        # self.phaseTwo(D0)
        print('\nEND!')

        #solve 
        self.render_process(epoch/nb_generations, ['Pop_size', 'Cost'], [[sum(self.nb_inds_tasks)], self.history_cost[-1]], use_sys= True)
        self.population.update_rank()
        self.last_pop = self.population
        return self.last_pop.get_solves()
    
    def reproduction(self, size: int, mating_pool: Population,) -> Population:
        sub_size = int(size/len(self.tasks))
       
        offsprings = Population(self.IndClass,
                                nb_inds_tasks = [0] * len(self.tasks), 
                                dim = self.dim_uss,
                                list_tasks= self.tasks)
        counter = np.zeros((len(self.tasks)))
        
        stopping = False
        while not stopping:
            pa, pb = mating_pool.__getRandomInds__(2)
            ta = pa.skill_factor
            tb = pb.skill_factor

            if counter[ta] >= sub_size and counter[tb] >= sub_size:
                continue

            rmpValue = numba_random_gauss(mean = max(self.rmp[ta][tb], self.rmp[tb][ta]), sigma = 0.1)

            if ta == tb:
                self.eval_k[ta] += 2

                oa, ob = self.crossover(pa, pb)

                oa.skill_factor = ta
                ob.skill_factor = ta
                
                offsprings.__addIndividual__(oa)
                offsprings.__addIndividual__(ob)

                counter[ta] += 2

            elif random.random() <= rmpValue:
                off = self.crossover(pa, pb)

                for o in off:
                    if counter[ta] < sub_size and random.random() < self.rmp[ta][tb]/(self.rmp[ta][tb] + self.rmp[tb][ta]):
                        o.skill_factor = ta
                        o = self.dimension_strategy(o, tb, pa)
                        o.fcost = self.tasks[ta](o)

                        offsprings.__addIndividual__(o)
                        
                        counter[ta] += 1
                        self.eval_k[ta] += 1
                        
                        if pa.fcost > o.fcost:
                            self.delta[ta][tb].append(pa.fcost - o.fcost)
                            self.s_rmp[ta][tb].append(rmpValue)
                    
                    elif counter[tb] < sub_size:
                        o.skill_factor = tb
                        o = self.dimension_strategy(o, ta, pb)
                        o.fcost = self.tasks[tb](o)

                        offsprings.__addIndividual__(o)
                        
                        counter[tb] += 1
                        self.eval_k[tb] += 1

                        if pb.fcost > o.fcost:
                            self.delta[tb][ta].append(pb.fcost - o.fcost)
                            self.s_rmp[tb][ta].append(rmpValue)

            else:
                if counter[ta] < sub_size:
                    paa: Individual = self.population[ta].__getRandomItems__()

                    # while np.array_equal(paa.genes, pa.genes):
                    #     paa: Individual = self.population[ta].__getRandomItems__()
                    
                    oa, _ = self.crossover(pa, paa)
                    oa.skill_factor = ta

                    offsprings.__addIndividual__(oa)

                    counter[ta] += 1
                    self.eval_k[ta] += 1

                if counter[tb] < sub_size:
                    pbb: Individual = self.population[tb].__getRandomItems__()

                    # while np.array_equal(pbb.genes, pb.genes):
                    #     pbb: Individual = self.population[tb].__getRandomItems__()
                    
                    ob, _ = self.crossover(pb, pbb)
                    ob.skill_factor = tb
                    
                    offsprings.__addIndividual__(ob)
                    
                    counter[tb] += 1
                    self.eval_k[tb] += 1
                    
            stopping = sum(counter >= sub_size) == len(self.tasks)

        return offsprings

    def phaseTwo(self, D0):
        fcosts = [sub.getFitness() for sub in self.population]

        D = self.calculateD(population = np.array([[ind.genes for ind in sub.ls_inds]for sub in self.population]), 
                            population_fitness = np.array(fcosts),
                            best = np.array([sub.__getBestIndividual__.genes for sub in self.population]),
                            )
        
        maxFit = np.max(fcosts, axis=1)
        minFit = np.min(fcosts, axis=1)
        maxDelta = maxFit - minFit + 1e-99

        assert len(D) == len(maxDelta), "Wrong shape. Got {} and {}".format(D.shape, maxDelta.shape)
        assert len(D) == len(self.tasks), "Got wrong shape"

        sigma = np.where(D > D0, 0, 1 - D/D0)
        nextPop = Population(IndClass = self.IndClass,
                            dim = self.dim_uss,
                            nb_inds_tasks=[0] * len(self.tasks),
                            list_tasks=self.tasks)
            
        for i in range(len(self.tasks)):
            self.eval_k[i] += self.learningPhase[i].evolve(self.population[i], nextPop, sigma[i], maxDelta[i])
        
        self.population = nextPop

    def calculateD(self, population: np.array, population_fitness: np.array, best: np.array) -> np.array:
        '''
        Arguments include:\n
        + `population`: genes of the current population
        + `population_fitness`: fitness of the current population
        + `best`: the best gene of each subpop
        + `nb_tasks`: number of tasks
        '''
        
        D = np.empty((len(self.tasks)))
        for i in range(len(self.tasks)):
            gene_max = [np.max(population[i], axis = 1).tolist()] * 50
            gene_min = [np.min(population[i], axis = 1).tolist()] * 50

            D[i] = self.__class__._calculateD(np.array(gene_max).T, np.array(gene_min).T, population[i], population_fitness[i], best[i], model.TOLERANCE)
        return D
    
    @jit(nopython = True, parallel = True, cache=True)
    def _calculateD(gene_max: np.array, gene_min: np.array, subPop: np.array, subPop_fitness: np.array, best: np.array, TOLERANCE: float) -> float:
            # gene_max = gene_max.flatten()
            # gene_max = np.broadcast_to(gene_max, (subPop.shape[-1], subPop.shape[0])).T
            # gene_min = gene_min.flatten()
            # gene_min = np.broadcast_to(gene_min, (subPop.shape[-1], subPop.shape[0])).T
            
            w = np.where(subPop_fitness > TOLERANCE, 1/(subPop_fitness), 1/TOLERANCE)
            # w = [1/ind if ind > TOLERANCE else 1/TOLERANCE for ind in population[i]]
            # print(subPop.shape)
            sum_w = sum(w)
            d = (subPop - gene_min)/(gene_max - gene_min)
            best = (best - gene_min)/(gene_max - gene_min)
            d = np.sum((d - best) ** 2, axis=1)
            d = np.sqrt(d)
            assert d.shape == w.shape
            # d = np.sqrt(np.sum(d, axis=0))
            # d = np.sum([np.sqrt(np.sum((d[i] - best) * (d[i] - best))) for i in range(len(subPop))])

            return np.sum(w * d/sum_w)
    
    def updateRMP(self, c: int):
        for i in range(len(self.tasks)):
            for j in range(len(self.tasks)):
                if i == j:
                    continue
                if len(self.delta[i][j]) > 0:
                    self.rmp[i][j] += self.__class__._updateRMP(self.delta[i][j], self.s_rmp[i][j], c)
                else:
                    self.rmp[i][j] = (1 - c) * self.rmp[i][j]
                
                self.rmp[i][j] = max(0.1, min(1, self.rmp[i][j]))

    @jit(nopython = True, parallel = True, cache= True)
    def _updateRMP(delta: List, s_rmp: List, c: float) -> float:
        delta = np.array(delta)
        s_rmp = np.array(s_rmp)
        sum_delta = sum(delta)
        tmp = (delta/sum_delta) * s_rmp
        meanS = sum(tmp * s_rmp)
        
        return c * meanS/sum(tmp)
    
class LearningPhase():
    M = 2
    H = 10
    def __init__(self, IndClass, list_tasks, task) -> None:
        self.IndClass = IndClass
        self.list_tasks = list_tasks
        self.task = task
        self.sum_improv = [0.0] * LearningPhase.M
        self.consume_fes = [0.0] * LearningPhase.M
        self.mem_cr = [0.5] * LearningPhase.H
        self.mem_f = [0.5] * LearningPhase.H
        self.s_cr = []
        self.s_f = []
        self.diff_f = []
        self.mem_pos = 0
        self.gen = 0
        self.best_opcode = 1
        self.searcher = [self.pbest1, PolynomialMutation(nm = 5).getInforTasks(self.IndClass, self.list_tasks)]

    def evolve(self, subPop: SubPopulation, nextPop: Population, sigma: float, max_delta: float) -> SubPopulation:
        self.gen += 1

        if self.gen > 1:
            self.best_opcode = self.__class__.updateOperator(sum_improve = self.sum_improv, 
                                                             consume_fes = self.consume_fes, 
                                                             M = LearningPhase.M)

            self.sum_improv = [0.0] * LearningPhase.M
            self.consume_fes = [1.0] * LearningPhase.M

        # self.updateMemory()
        
        pbest_size = max(5, int(0.15 * len(subPop)))
        # pbest = subPop.__getRandomItems__(size = pbest_size)
        idx_inds = np.argsort([ind.fcost for ind in subPop.ls_inds])
        pbest =  [subPop.ls_inds[i] for i in idx_inds[:pbest_size]] 

        for ind in subPop:
            r = random.randint(0, LearningPhase.M - 1)
            cr = numba_random_gauss(self.mem_cr[r], 0.1)
            f = numba_random_cauchy(self.mem_f[r], 0.1)

            opcode = random.randint(0, LearningPhase.M)
            if opcode == LearningPhase.M:
                opcode = self.best_opcode

            self.consume_fes[opcode] += 1
            
            if opcode == 0:
                child = self.searcher[opcode](ind, subPop, pbest, cr, f)
            elif opcode == 1:
                child = self.searcher[opcode](ind, return_newInd=True)

            child.skill_factor = ind.skill_factor
            child.fcost = self.task(child)
            
            diff = ind.fcost - child.fcost
            if diff > 0:
                survival = child

                self.sum_improv[opcode] += diff

                if opcode == 0:
                    self.diff_f.append(diff)
                    # self.s_cr.append(cr)
                    # self.s_f.append(f)
                
            elif diff == 0 or random.random() <= sigma * np.exp(diff/max_delta):
                survival = child
            else:
                survival = ind
            
            nextPop.__addIndividual__(survival)
        
        return pbest_size
    
    def pbest1(self, ind: Individual, subPop: SubPopulation, best: List[Individual], cr: float, f: float) -> Individual:
        pbest = best[random.randint(0, len(best) - 1)]
        
        ind_ran1, ind_ran2 = subPop.__getRandomItems__(size = 2, replace= False)
        
        u = (numba_random_uniform(len(ind.genes)) < cr)
        if np.sum(u) == 0:
            u = np.zeros(shape= (subPop.dim,))
            u[numba_randomchoice(subPop.dim)] = 1

        # new_genes = np.where(u, 
        #     pbest.genes + f * (ind_ran1.genes - ind_ran2.genes),
        #     ind.genes
        # )
        # # new_genes = np.clip(new_genes, ind.genes/2, (ind.genes + 1)/2)
        # new_genes = np.where(new_genes < 0, ind.genes/2, np.where(new_genes > 1, (ind.genes + 1)/2, new_genes))

        new_genes = self.__class__.produce_inds(ind.genes, pbest.genes, ind_ran1.genes, ind_ran2.genes, f, u)
        new_ind = self.IndClass(new_genes)

        return new_ind

    @jit(nopython=True, parallel = True)
    def produce_inds(ind_genes: np.array, best_genes: np.array, ind1_genes: np.array, ind2_genes: np.array, F: float, u: np.array) -> np.array:
        new_genes = np.where(u,
            best_genes + F * (ind1_genes - ind2_genes),
            ind_genes
        )
        new_genes = np.where(new_genes > 1, (ind_genes + 1)/2, new_genes) 
        new_genes = np.where(new_genes < 0, (ind_genes + 0)/2, new_genes)

        return new_genes

    # def updateMemory(self):
    #     if len(self.s_cr) > 0:
    #         # self.diff_f = np.array(self.diff_f)
    #         # self.s_cr = np.array(self.s_cr)
    #         # self.s_f = np.array(self.s_f)

    #         self.mem_cr[self.mem_pos] = self.__class__.updateMemoryCR(self.diff_f, self.s_cr)
    #         self.mem_f[self.mem_pos] = self.__class__.updateMemoryF(self.diff_f, self.s_f)
            
    #         self.mem_pos = (self.mem_pos + 1) % LearningPhase.H

    #         self.s_cr = []
    #         self.s_f = []
    #         self.diff_f = []

    # @jit(nopython = True, parallel = True, cache=True)
    # def updateMemoryCR(diff_f: List, s_cr: List) -> float:
    #     diff_f = np.array(diff_f)
    #     s_cr = np.array(s_cr)

    #     sum_diff = sum(diff_f)
    #     weight = diff_f/sum_diff
    #     tmp_sum_cr = sum(weight * s_cr)
    #     mem_cr = sum(weight * s_cr * s_cr)
        
    #     if tmp_sum_cr == 0 or mem_cr == -1:
    #         return -1
    #     else:
    #         return mem_cr/tmp_sum_cr
        
    # @jit(nopython = True, parallel = True, cache = True)
    # def updateMemoryF(diff_f: List, s_f: List) -> float:
    #     diff_f = np.array(diff_f)
    #     s_f = np.array(s_f)

    #     sum_diff = sum(diff_f)
    #     weight = diff_f/sum_diff
    #     tmp_sum_f = sum(weight * s_f)
    #     return sum(weight * (s_f ** 2)) / tmp_sum_f

    @jit(nopython = True, parallel = True)
    def updateOperator(sum_improve: List, consume_fes: List, M: int) -> int:
        sum_improve = np.array(sum_improve)
        consume_fes = np.array(consume_fes)
        eta = sum_improve / consume_fes
        best_rate = max(eta)
        best_op = np.argmax(eta)
        if best_rate > 0:
            return best_op
        else:
            return random.randint(0, M - 1)

In [27]:
baseModel = model()
# from pyMSOO.MFEA.model import EME_BI
# baseModel = EME_BI.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    # dimension_strategy= DimensionAwareStrategy.DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 03m 48.67s   99 % [===================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  2.26E-26  0.00E+00  0.00E+00  0.00E+00  4.31E-14  0.00E+00  6.36E-04  0.00E+00  7.96E+00  ,  
END!
Seed: None -- Time: 03m 50.91s  100 % [====================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  1.62E-26  0.00E+00  0.00E+00  0.00E+00  3.95E-14  0.00E+00  6.36E-04  0.00E+00  7.96E+00  ,  

In [25]:
print([lp.best_opcode for lp in baseModel.learningPhase])

[100015, 100010, 99998, 100002, 99994, 99999, 99997, 99998, 100000, 99999]

In [63]:

baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 03m 6.15s   99 % [===================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  1.70E-26  0.00E+00  0.00E+00  0.00E+00  3.60E-14  0.00E+00  6.36E-04  0.00E+00  0.00E+00  ,   
END!
Seed: None -- Time: 03m 7.76s  100 % [====================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  1.70E-26  0.00E+00  0.00E+00  0.00E+00  3.24E-14  0.00E+00  6.36E-04  0.00E+00  0.00E+00  ,  

In [28]:
import pandas as pd

df = pd.read_csv(r"F:\BTVN\DSAI\Optimization Lab\Paper\Efficient knowledge transfer\history_cost_summaries_EME_BI_woDaS.csv")

In [29]:
df.tail(50)

,Unnamed: 0,EME-BI,EME_BI_wo_DaS
450,450,0.000000,0.649250
451,451,153.862380,140.877084
452,452,0.000000,0.000329
453,453,10.890937,23.687499
454,454,5022.383910,6610.532774
455,455,0.000000,8.183991
456,456,112.875490,159.721696
457,457,0.000000,0.000000
458,458,12.390112,21.295734
459,459,4979.867545,6728.074941


In [30]:
# from pyMSOO.MFEA.model import EME_BI

# ls_benchmark = []
# ls_IndClass = []
# name_benchmark = []
# ls_tasks = [10]

# for i in ls_tasks:
#     # t, ic = WCCI22_benchmark.get_complex_benchmark(i)
#     t, ic = WCCI22_benchmark.get_50tasks_benchmark(i)
#     ls_benchmark.append(t)
#     ls_IndClass.append(ic)
#     name_benchmark.append(str(i))



# smpModel = MultiBenchmark(
#     ls_benchmark= ls_benchmark,
#     name_benchmark= name_benchmark,
#     ls_IndClass= ls_IndClass,
#     model= EME_BI
# )

# smpModel.compile(
#     crossover= SBX_Crossover(nc = 2),
#     mutation= PolynomialMutation(nm = 5),
#     selection= ElitismSelection(),
#     # dimension_strategy = DaS_strategy(eta = 3)
# )
# smpModel.fit(
#     nb_generations = 1000,nb_inds_each_task= 100, 
#     bound_pop= [0, 1], evaluate_initial_skillFactor= True
# )
# a = smpModel.run(
#     nb_run= 1)

In [43]:
res = []
# for i in range(1, 11):
t, ic = WCCI22_benchmark.get_50tasks_benchmark(10)
baseModel = model()
baseModel.compile(
    IndClass= ic,
    tasks= t,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    # dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

# res.append([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

Seed: None -- Time: 24m 52.38s   99 % [===================>]  Pop_size: 1.00E+03  ,  Cost: 1.99E+01  8.47E+01  0.00E+00  8.18E-02  5.69E+03  2.00E+01  5.28E+01  0.00E+00  7.67E+00  7.61E+03  2.53E-14  6.87E+01  0.00E+00  1.66E+00  5.69E+03  2.53E-14  8.83E+01  0.00E+00  1.30E+01  7.32E+03  2.89E-14  8.81E+01  0.00E+00  2.59E-01  4.86E+03  2.00E+01  4.88E+01  1.11E-16  6.98E+00  7.25E+03  1.99E+01  6.53E+01  1.11E-16  1.09E+01  4.46E+03  2.00E+01  8.13E+01  0.00E+00  1.58E+00  6.76E+03  2.89E-14  9.96E+01  0.00E+00  1.97E+00  6.21E+03  2.00E+01  6.24E+01  0.00E+00  6.19E-02  4.94E+03  ,  
END!
Seed: None -- Time: 25m 5.04s  100 % [====================>]  Pop_size: 1.00E+03  ,  Cost: 1.99E+01  8.44E+01  0.00E+00  8.18E-02  5.69E+03  2.00E+01  5.25E+01  0.00E+00  7.67E+00  7.61E+03  2.89E-14  6.87E+01  0.00E+00  1.66E+00  5.69E+03  2.89E-14  8.82E+01  0.00E+00  1.30E+01  7.32E+03  2.89E-14  8.54E+01  0.00E+00  2.59E-01  4.86E+03  2.00E+01  4.88E+01  1.11E-16  6.98E+00  7.25E+03  1.99E+01 

In [44]:
h = np.array([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

In [45]:
h = np.where(h < 1e-6, 0, h)

In [46]:
print(h)
print(np.array(df["EME-BI"][-50:]))

[1.99465199e+01 8.43881889e+01 0.00000000e+00 8.18242387e-02
 5.68618259e+03 1.99714880e+01 5.24616162e+01 0.00000000e+00
 7.67451108e+00 7.61248528e+03 0.00000000e+00 6.86824019e+01
 0.00000000e+00 1.66300205e+00 5.69074145e+03 0.00000000e+00
 8.82043227e+01 0.00000000e+00 1.30255117e+01 7.31935204e+03
 0.00000000e+00 8.52809900e+01 0.00000000e+00 2.59298663e-01
 4.85861752e+03 1.99765960e+01 4.88456755e+01 0.00000000e+00
 6.98306116e+00 7.24676037e+03 1.99238871e+01 6.52713702e+01
 0.00000000e+00 1.09407466e+01 4.46212616e+03 1.99816756e+01
 8.12682932e+01 0.00000000e+00 1.58228123e+00 6.75956456e+03
 0.00000000e+00 9.92276683e+01 0.00000000e+00 1.96977725e+00
 6.20852231e+03 1.99561195e+01 6.23440343e+01 0.00000000e+00
 6.19344617e-02 4.94269582e+03]
[0.00000000e+00 1.53862380e+02 0.00000000e+00 1.08909369e+01
 5.02238391e+03 0.00000000e+00 1.12875490e+02 0.00000000e+00
 1.23901121e+01 4.97986754e+03 2.99928333e-02 1.28012455e+02
 0.00000000e+00 1.26848136e+01 5.05006055e+03 2.06536

In [47]:
sum(h > df["EME-BI"][-50:])

15

In [48]:
sum(h < df["EME-BI"][-50:])

24

In [50]:
sum(h < f)

22

In [51]:
sum(h > f)

18

In [37]:
res = []
# for i in range(1, 11):
t, ic = WCCI22_benchmark.get_50tasks_benchmark(10)
baseModel = model()
baseModel.compile(
    IndClass= ic,
    tasks= t,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

# res.append([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

Seed: None -- Time: 26m 45.56s   99 % [===================>]  Pop_size: 1.00E+03  ,  Cost: 2.89E-14  7.86E+01  0.00E+00  4.19E+00  6.45E+03  2.00E+01  6.53E+01  1.67E-15  3.14E+00  5.60E+03  1.99E+01  6.65E+01  0.00E+00  6.58E+00  7.72E+03  2.00E+01  7.61E+01  0.00E+00  8.31E+00  6.24E+03  1.99E+01  9.50E+01  1.11E-16  1.55E-01  5.08E+03  2.89E-14  1.12E+02  0.00E+00  1.21E+01  4.96E+03  2.89E-14  6.34E+01  0.00E+00  3.82E+00  7.41E+03  1.99E+01  5.82E+01  0.00E+00  9.44E+00  5.25E+03  2.00E+01  6.90E+01  0.00E+00  3.47E+00  8.15E+03  2.00E+01  6.28E+01  0.00E+00  3.52E+00  6.84E+03  ,  
END!
Seed: None -- Time: 26m 58.97s  100 % [====================>]  Pop_size: 1.00E+03  ,  Cost: 2.89E-14  7.86E+01  0.00E+00  4.19E+00  6.45E+03  2.00E+01  6.53E+01  1.67E-15  3.14E+00  5.60E+03  1.99E+01  6.64E+01  0.00E+00  6.58E+00  7.72E+03  2.00E+01  7.48E+01  0.00E+00  8.31E+00  6.24E+03  1.99E+01  9.40E+01  1.11E-16  1.55E-01  5.08E+03  3.24E-14  1.12E+02  0.00E+00  1.21E+01  4.96E+03  2.89E-14

In [38]:
f = np.array([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

In [39]:
print(f)
print(np.array(df["EME-BI"][-50:]))

[2.88657986e-14 7.86213722e+01 0.00000000e+00 4.19149613e+00
 6.44892467e+03 1.99815250e+01 6.52893227e+01 1.66533454e-15
 3.14240182e+00 5.59822794e+03 1.99448631e+01 6.64177645e+01
 0.00000000e+00 6.58146619e+00 7.72265685e+03 1.99675411e+01
 7.47662851e+01 0.00000000e+00 8.31223049e+00 6.24413039e+03
 1.99320109e+01 9.39396401e+01 1.11022302e-16 1.54717625e-01
 5.07698397e+03 2.88657986e-14 1.11911718e+02 0.00000000e+00
 1.20604467e+01 4.95923053e+03 2.88657986e-14 4.38322588e+01
 0.00000000e+00 3.81593582e+00 7.40915177e+03 1.99358046e+01
 5.82412989e+01 0.00000000e+00 9.44232361e+00 5.24641177e+03
 1.99903940e+01 6.89739982e+01 0.00000000e+00 3.46751251e+00
 8.14606994e+03 1.99709921e+01 6.27447499e+01 0.00000000e+00
 3.52363522e+00 6.83571201e+03]
[0.00000000e+00 1.53862380e+02 0.00000000e+00 1.08909369e+01
 5.02238391e+03 0.00000000e+00 1.12875490e+02 0.00000000e+00
 1.23901121e+01 4.97986754e+03 2.99928333e-02 1.28012455e+02
 0.00000000e+00 1.26848136e+01 5.05006055e+03 2.06536

In [40]:
f = np.where(f < 1e-6, 0, f)

In [41]:
sum(f < df["EME-BI"][-50:])

22

In [42]:
sum(f > df["EME-BI"][-50:])

17

In [49]:
sum(f < df["EME_BI_wo_DaS"][-50:])

33

In [62]:
sum(df["EME_BI_wo_DaS"][-50:] < df["EME-BI"][-50:])

2

In [52]:
sum(df["EME_BI_wo_DaS"][-50:] > df["EME-BI"][-50:])

43